# DELAY SUM - DREGON DATASET

A ideia desse Jupyter é usar o dataset Dregon para aplicar o algoritmo Delay Sum.
Segundo o dataset, http://dregon.inria.fr/datasets/dregon/, as coordenadas X, Y e Z em metros dos oito microfones são:

mic0 = [0.0420, 0.0615, -0.0410]

mic1 = [-0.0420, 0.0615, 0.0410]

mic2 = [-0.0615, 0.0420, -0.0410]

mic3 = [-0.0615, -0.0420, 0.0410]

mic4 = [-0.0420, -0.0615, -0.0410]

mic5 = [0.0420, -0.0615, 0.0410]

mic6 = [0.0615, -0.0420, -0.0410]

mic7 = [0.0615, 0.0420, 0.0410]

In [1]:
import librosa
import matplotlib.pyplot as plt
from IPython.display import Audio
import math
import numpy as np

### Importando um áudio qualquer

Segundo o dataset, os áudios são nomeados como se segue:

XX_YY_ZZ.wav where XX denotes the relative azimuth, YY the relative elevation and ZZ the relative distance of the loudspeaker from the array.

O dataset foi gravado com 44100 Hz, e, como o librosa importa os áudios sempre com 22kHz, é preciso colocar sr=None como parâmetro adicional da função abaixo.

In [2]:
nomeArquivo = '75_-30_2.4__2.wav'

In [3]:
originalAudio, freqAmostragem = librosa.load('/home/dimi/Downloads/datasets/DREGON_clean_recordings_speech/' + nomeArquivo, sr=None, mono=False)

In [4]:
freqAmostragem

44100

Ouvindo os microfones que serão utilizados (mesmo eixo Z):

In [5]:
Audio(data=originalAudio[1], rate=freqAmostragem)

In [6]:
Audio(data=originalAudio[3], rate=freqAmostragem)

In [7]:
Audio(data=originalAudio[5], rate=freqAmostragem)

In [8]:
Audio(data=originalAudio[7], rate=freqAmostragem)

### Função para verificar o delay entre dois sinais pela correlação

Essa função foi copiada e colada do algoritmo que já havia sido desenvolvido antes. É preciso mandar o argumento maxDelay, que é a quantidade de amostras contidas no maior delay possível entre dois microfones quaisquer.

In [9]:
def verificarDelay(sinalA, sinalB, maxDelay=15, verbose=False):
    
    # Verificando se os dois sinais tem o mesmo tamanho e se maxDelay é compatível
    if len(sinalA) != len(sinalB) or maxDelay >= len(sinalA)-1 :
        return False
    
    # Sei que o maior delay possível (em qtd de amostras) é maxDelay. Portanto, em cada iteração, 
    # a comparação entre os sinais para gerar a correlação se dará com arrays de tamanho 
    # len(sinal) - maxDelay
    tamanho = len(sinalA) - maxDelay
    
    # O delay será a qtd de amostras pra trás ou para frente que há
    # entre o sinal B em relação ao sinal A. Negativo significa que B está adiantado, 0 que não há
    # delay e positivo que está atrasado.
    delay = 0
    melhorCorrelacao = -1
    arrayCorrelacoes = []
    
    inicioA = 0
    fimA    = inicioA + tamanho
    inicioB = len(sinalB) - tamanho
    fimB    = inicioB + tamanho
    
    # Fazendo as iterações e calculando a correlação entre os sinais
    for i in range(-maxDelay, maxDelay+1):
        
        # Calculando a correlação da iteração atual
        corrAtual = np.corrcoef(sinalA[inicioA:fimA], sinalB[inicioB:fimB])[0][1]
        
        # Verificando se encontramos uma correlacao maior ainda para atualizar o delay
        if corrAtual > melhorCorrelacao:
            melhorCorrelacao = corrAtual
            delay = -i
        
        # Printando as iterações se verbose estiver ligado
        if verbose == True:
            arrayCorrelacoes.append(corrAtual)
            plt.title("Delay " + str(i))
            plt.plot(sinalA[inicioA:fimA])
            plt.plot(sinalB[inicioB:fimB])
            plt.show()
            print("Sinal A:", int(inicioA), "até", int(fimA))
            print("Sinal B:", int(inicioB), "até", int(fimB))
            print("Correlação Atual:", corrAtual)
            print("Melhor Correlação:", melhorCorrelacao, "(Delay " + str(delay) + ")\n\n\n")
            
        # Fazendo os indexes dos arrays da próxima iteração. De i = -maxDelay até 0, o Sinal A 
        # fica parado e o Sinal B vem vindo pra trás. A partir de i = 1 até maxDelay, o Sinal B
        # fica parado e o Sinal A vai embora.
        if i < 0:
            inicioB -= 1
            fimB     = inicioB + tamanho
        else:
            inicioA += 1
            fimA     = inicioA + tamanho
         
    # Se o verbose estiver ligado eu printo as correlações em função do delay tb
    if verbose == True:
        plt.title("Correlações em função dos delays")
        plt.plot(arrayCorrelacoes)
        plt.show()
    
    # Retornando o delay
    return -delay

A função definirDelayMaximo pode calcular a quantidade de amostras do maior delay possível entre dois microfones. Basta enviar o tamanho do lado do cubo.

In [10]:
def definirDelayMaximo(freqAmostragem, L, velocidadeSom=340):
    
    # O delay máximo (em qtd de amostras) dar-se-á na diagonal dos microfones
    diagonal     = math.sqrt(2 * (L**2))
    tempo        = diagonal / velocidadeSom
    qtdAmostras  = freqAmostragem * tempo
    
    # Adicionando mais 1 amostra pra garantir
    return int(qtdAmostras + 1)

### Verificando o funcionamento da função calculando o delay entre os microfones

In [11]:
coordenadasMic0 = [0.0420, 0.0615, -0.0410]
coordenadasMic1 = [-0.0420, 0.0615, 0.0410]
coordenadasMic2 = [-0.0615, 0.0420, -0.0410]
coordenadasMic3 = [-0.0615, -0.0420, 0.0410]
coordenadasMic4 = [-0.0420, -0.0615, -0.0410]
coordenadasMic5 = [0.0420, -0.0615, 0.0410]
coordenadasMic6 = [0.0615, -0.0420, -0.0410]
coordenadasMic7 = [0.0615, 0.0420, 0.0410]

vetorLado = [coordenadasMic1[0] - coordenadasMic3[0], coordenadasMic1[1] - coordenadasMic3[1], coordenadasMic1[2] - coordenadasMic3[2]]
tamanhoLado = math.sqrt(vetorLado[0]**2 + vetorLado[1]**2 + vetorLado[2]**2)

delayMax = definirDelayMaximo(freqAmostragem, L=tamanhoLado)
print(delayMax)

20


Calculando o delay entre os microfones tendo o mic0 como referência.

In [12]:
delay01 = verificarDelay(originalAudio[0], originalAudio[1], maxDelay=delayMax)
delay02 = verificarDelay(originalAudio[0], originalAudio[2], maxDelay=delayMax)
delay03 = verificarDelay(originalAudio[0], originalAudio[3], maxDelay=delayMax)
delay04 = verificarDelay(originalAudio[0], originalAudio[4], maxDelay=delayMax)
delay05 = verificarDelay(originalAudio[0], originalAudio[5], maxDelay=delayMax)
delay06 = verificarDelay(originalAudio[0], originalAudio[6], maxDelay=delayMax)
delay07 = verificarDelay(originalAudio[0], originalAudio[7], maxDelay=delayMax)

print(delay01)
print(delay02)
print(delay03)
print(delay04)
print(delay05)
print(delay06)
print(delay07)

-8
-6
-20
-17
-20
-11
-8


## Implementando o produto interno

http://www.labbookpages.co.uk/audio/beamforming/delayCalc.html

### Passando a origem pra cima do microfone 1

In [13]:
coordenadasMic0 = np.array(coordenadasMic0)
coordenadasMic1 = np.array(coordenadasMic1)
coordenadasMic2 = np.array(coordenadasMic2)
coordenadasMic3 = np.array(coordenadasMic3)
coordenadasMic4 = np.array(coordenadasMic4)
coordenadasMic5 = np.array(coordenadasMic5)
coordenadasMic6 = np.array(coordenadasMic6)
coordenadasMic7 = np.array(coordenadasMic7)

coordenadasNovasMic0 = coordenadasMic0 - coordenadasMic0
coordenadasNovasMic1 = coordenadasMic1 - coordenadasMic0
coordenadasNovasMic2 = coordenadasMic2 - coordenadasMic0
coordenadasNovasMic3 = coordenadasMic3 - coordenadasMic0
coordenadasNovasMic4 = coordenadasMic4 - coordenadasMic0
coordenadasNovasMic5 = coordenadasMic5 - coordenadasMic0
coordenadasNovasMic6 = coordenadasMic6 - coordenadasMic0
coordenadasNovasMic7 = coordenadasMic7 - coordenadasMic0

print(coordenadasNovasMic0)
print(coordenadasNovasMic1)
print(coordenadasNovasMic2)
print(coordenadasNovasMic3)
print(coordenadasNovasMic4)
print(coordenadasNovasMic5)
print(coordenadasNovasMic6)
print(coordenadasNovasMic7)

[0. 0. 0.]
[-0.084  0.     0.082]
[-0.1035 -0.0195  0.    ]
[-0.1035 -0.1035  0.082 ]
[-0.084 -0.123  0.   ]
[ 0.    -0.123  0.082]
[ 0.0195 -0.1035  0.    ]
[ 0.0195 -0.0195  0.082 ]


### Resolvendo o sistema com mínimos quadrados

https://riptutorial.com/numpy/example/16034/find-the-least-squares-solution-to-a-linear-system-with-np-linalg-lstsq

#### Exemplo de como usar a função

In [14]:
# Definindo a equação matricial
A = np.array([[1,0,0,0],
              [0,1,0,0],
              [0,0,1,0],
              [0,0,0,1]])
b = np.array([1,2,3,4])

# Resolvendo x
x, residuals, rank, s = np.linalg.lstsq(A,b, rcond=None)

print(x)

[1. 2. 3. 4.]


#### Aplicando no problema dos microfones

A matriz dos coeficientes será as coordenadas dos microfones 3, 5 e 7 e a matriz b será os delays multiplicados pela velocidade do som.

In [15]:
A = np.array([
    coordenadasNovasMic1.tolist(),
    coordenadasNovasMic2.tolist(),
    coordenadasNovasMic3.tolist(),
    coordenadasNovasMic4.tolist(),
    coordenadasNovasMic5.tolist(),
    coordenadasNovasMic6.tolist(),
    coordenadasNovasMic7.tolist()
])
b = np.array([
    (delay01/freqAmostragem)*340,
    (delay02/freqAmostragem)*340,
    (delay03/freqAmostragem)*340,
    (delay04/freqAmostragem)*340,
    (delay05/freqAmostragem)*340,
    (delay06/freqAmostragem)*340,
    (delay07/freqAmostragem)*340
])

w, residuals, rank, s = np.linalg.lstsq(A,b, rcond=None)
print(w)

[ 0.2332128   0.88349241 -0.53422911]


### Definindo algumas funções pra ajudar

In [16]:
def vetorUnitario(vetor):
    return vetor/np.linalg.norm(vetor)

def anguloEntreDoisVetores(v1, v2):
    v1Unitario = vetorUnitario(v1)
    v2Unitario = vetorUnitario(v2)
    return np.arccos(np.clip(np.dot(v1Unitario, v2Unitario), -1.0, 1.0))

def anguloAzimutal(vetor):
    return np.arctan(vetor[1]/vetor[0])
    
def anguloElevacao(vetor):
    return anguloEntreDoisVetores([vetor[0], vetor[1], 0], vetor)

def radParaGrau(angulo):
    return (angulo*180)/math.pi

In [17]:
print("Ângulo Azimutal:", radParaGrau(anguloAzimutal(w)))
print("Ângulo de Elevação:", radParaGrau(anguloElevacao(w)))

Ângulo Azimutal: 75.21309254061818
Ângulo de Elevação: 30.312813661385682
